In [1]:
from pderivative import partialDerivative
from pderivative import multiOut
from pderivative import grad
from multivar_optimization import multiGrad
from math import *

In [2]:
def multiGradConstraint(func,X,Y,constraint,option = None):
    """
    task: calculate the optimum point for a multivariable function: F = F(x1,x2,...,xn)
    func: objective function (str)
    X: list of variables (strings) {x1,x2,...,xn}
    Y: initial guess for the optimum value (list of int/floats)
    constraint: [[x1,x2],[y1,y2]]
    option: choose whether you want to solve maxima or minima. default option: minima
    
    output: {solution, mean error} (dict)
    """
    solutionC = list()
    extrema = list()
    c = None
    criteria = True
    
    if (option == None) or (option.lower() == 'minima'):
        solution = multiGrad(func,X,Y)['optimum solution: ']
        
        if len(constraint) == len(solution):
            for k in range(0,len(constraint)):
                if (solution[k] >= constraint[k][0]) and (solution[k] <= constraint[k][1]):
                    solutionC.append(solution[k])
                else:
                    criteria = False
                    
        else:
            raise ValueError("dim(constraint) is not equal to dim(solution)")
    
    if (option.lower() == 'maxima'):
        solution = multiGrad(func,X,Y,option = 'maxima')['optimum solution: ']
        
        if len(constraint) == len(solution):
            for k in range(0,len(constraint)):
                if (solution[k] >= constraint[k][0]) and (solution[k] <= constraint[k][1]):
                    solutionC.append(solution[k])
                else:
                    criteria = False
        
        else:
            raise ValueError("dim(constraint) is not equal to dim(solution)")
    
    if (criteria == False) and (option == None or option.lower() == 'minima'):
        print("There is no local minima within set of constraints established.")
    
    elif (criteria == False) and (option.lower() == 'maxima'):
        print("There is no local maxima within set of constraints established.")
    
    elif (criteria == True):
        return solutionC
    

In [3]:
#code shows constrained optimization case works for f(X,Y) = 50-x**2-2*y**2
#constraints: {-5≤X≤5}, {-5≤Y≤5}
multiGradConstraint('50-x**2-2*y**2',['x','y'],[1,1],[[-5,5],[-5,5]],option = 'maxima')

[3.471964282653062e-05, 1.7211869817401748e-05]

In [4]:
#code shows constrained optimization case fails for f(X,Y) = 50-x**2-2*y**2, as local maxima is at (0,0)
#constraints: {10≤X≤20}, {10≤Y≤20}
multiGradConstraint('50-x**2-2*y**2',['x','y'],[1,1],[[10,20],[10,20]],option = 'maxima')

There is no local maxima within set of constraints established.


In [5]:
def Jacobian(func,X,Y):
    """
    task: output jacobian matrix of f = [f1(x1,...,xn),f2(x1,...,xn),...,fm(x1,...,xn)]
    func: set of multivariate functions
    X: set of string variables in list: [x1,x2,...,xn]
    Y: point of evaluation in list (integers in list)
    dim(X) = dim(Y)
    
    output: Jacobian matrix; J(f); dim(J) = (m x n)
    """
    J = []
    m = len(func)
    n = len(X)
    f = None
    
    for k in range(0,m):
        f = grad(func[k],X,Y)
        J.append(f)
    
    #error check:
    if (len(J) == m) and (len(J[0]) == len(X)):
        return J
    else:
        raise ValueError("The dimensions of the Jacobian Matrix are inconsistent")

In [6]:
func = ['x**2+y**2','50-x**2-2*y**2','sin(x*y)']
X = ['x','y']
Y = [1,1]
Jacobian(func,X,Y)

[[2.000000165480742, 2.000000165480742],
 [-2.00003569261753, -4.000000330961484],
 [0.5403022473871033, 0.5403022473871033]]

In [20]:
def secondPartial(func,X,Y,diff1,diff2):
    """
    func -> objective function (str)
    X: list of string variables of interest (list)
    Y: point of evaluation (list)
    diff1: differentiate with respect to this variable (str)
    diff2: differentiate a second time with respect to this variable (str)
    
    output: second partial derivative
    lim h->0 (f(x1,x2,..,(xL+h),..,(xU+h),..,xN) - f(x1,x2,..,(xL+h),..,xU,..xN) - f(x1,..,xL,..,(xU+h),..,xN) + f(
    x1,x2,...,xN)/(h**2)
    """
    
    math_ops = {'sqrt':sqrt,'log':log,'sin':sin,'cos':cos,'tan':tan,
                'asin':asin, 'acos':acos, 'atan':atan, 'asinh':asinh, 'acosh':acosh,
                'atanh':atanh, 'pow':pow, 'exp':exp,
                'fabs':fabs, 'factorial': factorial, 'floor': floor}
    
    h = 0.00001
    D = {}
    D1 = {}
    D2 = {}
    D3 = {}
    
    if len(X) == len(Y):
        if diff1 != diff2:
            #MAPPING: X[i] -> Y[i]:
            for k in range(0,len(X)):
                D[X[k]] = Y[k] #{'x1':A1, 'x2': A2, ...., 'xN':AN}

            seg1 = eval(func,D,math_ops) #f(x1,x2,....,xN)
            
            #f(x1,x2,...,(xL+h),..,(xU+h),...,xN)
            for k in D.keys():
                if (k == diff1) or (k == diff2):
                    D1[k] = D[k] + h
                else:
                    D1[k] = D[k]
            
            seg2 = eval(func,D1,math_ops)
            
            #f(x1,x2,...,(xL+h),...,xU,...,xN)
            for k in D.keys():
                if (k == diff1):
                    D2[k] = D[k] + h
                else:
                    D2[k] = D[k]
            
            seg3 = eval(func, D2, math_ops)
            
            #f(x1,x2,...,xL,...,(xU+h),...,xN)
            for k in D.keys():
                if (k == diff2):
                    D3[k] = D[k] + h
                else:
                    D3[k] = D[k]
            
            seg4 = eval(func,D3,math_ops)
            
            return (seg2 - seg3 - seg4 + seg1)/(h**2)
        
        elif (diff1 == diff2):
            
            for k in range(0,len(X)):
                D[X[k]] = Y[k] #{'x1':A1, 'x2': A2, ...., 'xN':AN}

            seg1 = eval(func,D,math_ops) #f(x1,x2,....,xN)
            
            #f(x1,x2,...,(xL+h),...,xN)
            for k in D.keys():
                if (k == diff1):
                    D1[k] = D[k] + h
                else:
                    D1[k] = D[k]
            
            seg2 = eval(func,D1,math_ops)
            
            #f(x1,x2,...,(xL+2h),...,xU,...,xN)
            for k in D.keys():
                if (k == diff1):
                    D2[k] = D[k] + 2*h
                else:
                    D2[k] = D[k]
            
            seg3 = eval(func, D2, math_ops)
            
            return (seg3 - 2*seg2 + seg1)/(h**2)
    
    else:
        raise ValueError("dim(X) is not equal to dim(Y).")

In [22]:
secondPartial('log(x*y)', ['x','y'],[1,1],'x','y')

8.273817828780004e-08

In [23]:
def secondGrad(func,X,Y,diff1):
    """
    input: objective function, func (str)
    X: list of variables (list of str)
    Y: point of evaluation (list of int/float)
    
    this is required to construct the Hessian matrix
    for f = f(x1,x2,...,xN):
    [d2f/d(diff*x1), d2f/d(diff*x2), ..., d2f/d(diff*diff),..., d2f/d(diff*xN)]
    """
    
    math_ops = {'sqrt':sqrt,'log':log,'sin':sin,'cos':cos,'tan':tan,
                'asin':asin, 'acos':acos, 'atan':atan, 'asinh':asinh, 'acosh':acosh,
                'atanh':atanh, 'pow':pow, 'exp':exp,
                'fabs':fabs, 'factorial': factorial, 'floor': floor}
    
    sGrad = []
    
    for k in range(0,len(X)):
        s = secondPartial(func,X,Y,diff1,X[k])
        sGrad.append(s)
    
    return sGrad

In [24]:
secondGrad('log(x*y)',['x','y'],[1,1],'x')

[-0.9999822207467484, 8.273817828780004e-08]

In [25]:
def Hessian(func,X,Y):
    """
    task: construct Hessian matrix
    input: objective function, func (str)
    X: list of variables (list of str)
    Y: point of evaluation (list of int/float)
    
    output: Hessian matrix: [[A11,A12..,A1N],[A21,A22,....,A2N]]
    """
    
    math_ops = {'sqrt':sqrt,'log':log,'sin':sin,'cos':cos,'tan':tan,
                'asin':asin, 'acos':acos, 'atan':atan, 'asinh':asinh, 'acosh':acosh,
                'atanh':atanh, 'pow':pow, 'exp':exp,
                'fabs':fabs, 'factorial': factorial, 'floor': floor}
    
    hessian = []
    
    for k in range(0,len(X)):
        D = X[k]
        hessian.append(secondGrad(func,X,Y,D))
    
    return hessian

In [27]:
func = 'x**3 - 2*x*y - y**6'
X = ['x','y']
Y = [1,2]
Hessian(func,X,Y)

[[6.000107077852589, -1.9998935840703778],
 [-1.9998935840703778, -480.0095609880372]]